# Random Background Analysis

In [34]:
### header ###
__author__ = "Jenhan Tao"
__license__ = "BSD"
__email__ = "jenhantao@gmail.com"

### imports ###
import sys
import os
import pandas as pd
import numpy as np
import argparse
import matplotlib
import itertools
import scipy
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy
import pickle
from sklearn import preprocessing
import sklearn
from sklearn import decomposition
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn import svm, datasets
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc
from sklearn import ensemble
from sklearn import neighbors
import matplotlib_venn
from sklearn.cross_validation import train_test_split
from random import shuffle

### notebook specific configuration ###
%matplotlib inline
matplotlib.pylab.rcParams['savefig.dpi'] = 200
sys.setrecursionlimit(5000)
os.chdir('/gpfs/data01/glasslab/home/jtao/analysis/random_background_analysis/')
sns.set_context('notebook')
%load_ext autoreload
%autoreload 2
%env PATH=/gpfs/data01/glasslab/home/jtao/perl5/bin:/gpfs/data01/glasslab/home/jtao/software/anaconda3/bin:/home/jtao/software/bin:/usr/local/sbin:/usr/sbin:/usr/bin:/usr/local/bin:/usr/bin:/gpfs/data01/glasslab/home/jtao/software/homer/bin:/gpfs/data01/glasslab/home/jtao/software/weblogo:/home/jtao/code/seq_merge_pipe:/home/vlink/mouse_strains/marge/shifting:/bioinformatics/glassutils/scripts:/bioinformatics/software/meme/bin:/home/jtao/software/lsgkm/bin

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: PATH=/gpfs/data01/glasslab/home/jtao/perl5/bin:/gpfs/data01/glasslab/home/jtao/software/anaconda3/bin:/home/jtao/software/bin:/usr/local/sbin:/usr/sbin:/usr/bin:/usr/local/bin:/usr/bin:/gpfs/data01/glasslab/home/jtao/software/homer/bin:/gpfs/data01/glasslab/home/jtao/software/weblogo:/home/jtao/code/seq_merge_pipe:/home/vlink/mouse_strains/marge/shifting:/bioinformatics/glassutils/scripts:/bioinformatics/software/meme/bin:/home/jtao/software/lsgkm/bin


## Functions

In [243]:
# split data into GC content matched training and test data
def get_GC_matched_split(features, labels, test_size, tolerance = 0.01):
    '''
    feature: 2D array (samples x features)
    labels: 1D boolean array (samples x)
    test_size: fraction of data to test on
    tolerance: max difference in GC content between True and False labelled samples
    '''
    global _id_sequence_dict
    
    ### match GC content of samples labelled True with those labelled False by thowing out False samples
    # retrieve sequences using index of labels
    index_label_tuples = tuple(zip(labels.index.values, labels.values))
    
    true_sequences = [_id_sequence_dict[x[0]] for x in index_label_tuples if x[1]]
    true_ids = [x[0] for x in index_label_tuples if x[1]]
    
    false_sequences = [_id_sequence_dict[x[0]] for x in index_label_tuples if not x[1]]
    false_ids = [x[0] for x in index_label_tuples if not x[1]]
    
    # calculate GC content of True samples
    true_gc_count = 0
    true_length = 0
    for s in true_sequences:
        true_gc_count += s.count('G')
        true_gc_count += s.count('C')
        true_length += len(s)
    true_gc_content = true_gc_count/(true_length+0.0000001)
    
    # calcuate GC content of False samples
    false_gc_count = 0
    false_length = 0
    for s in false_sequences:
        false_gc_count += s.count('G')
        false_gc_count += s.count('C')
        false_length += len(s)
    false_gc_content = false_gc_count/(false_length+0.0000001)
    
    while abs(true_gc_content - false_gc_content) > tolerance:
        # remove false GC sequences until GC content matches tolerance
        selected_seq = False
        
        while not selected_seq:
            rand_index = np.random.randint(len(false_sequences))
            current_seq = false_sequences[rand_index]
            current_gc_count = current_seq.count('G')+ current_seq.count('C')
            current_length = len(current_seq)
            current_gc = current_gc_count/current_length
            if true_gc_content > false_gc_content:
                # remove sequences that would increase overall GC content of False sequences
                if current_gc < false_gc_content:
                    selected_seq = True
            else:
                # remove sequences that would decrease overall GC content of False sequences
                if current_gc > false_gc_content:
                    selected_seq = True
        false_gc_count -= current_gc_count
        false_length -= current_length
        false_gc_content = false_gc_count/false_length
        
        false_sequences.pop(rand_index)
        false_ids.pop(rand_index)
    
    filtered_ids = true_ids + false_ids
    filtered_features = features[features.index.isin(filtered_ids)]
    filtered_labels = labels[labels.index.isin(filtered_ids)]

    if test_size <= 0.5:
        training_indices, test_indices = next(iter(
                sklearn.cross_validation.StratifiedKFold(filtered_labels, int(1/test_size), shuffle=True)))
    else:
        test_indices, training_indices = next(
            iter(sklearn.cross_validation.StratifiedKFold(filtered_labels, int(1/(1-test_size)), shuffle=True)))
    training_ids = [filtered_ids[i] for i in training_indices]
    test_ids = [filtered_ids[i] for i in test_indices]
    
    training_features = filtered_features[filtered_features.index.isin(training_ids)]
    test_features = filtered_features[filtered_features.index.isin(test_ids)]
    training_labels = filtered_labels[filtered_labels.index.isin(training_ids)]
    test_labels = filtered_labels[filtered_labels.index.isin(test_ids)]
    
    return training_features, test_features, training_labels, test_labels
    

In [244]:
# split data into GC content matched training and test data
def get_split(features, labels, test_size):
    '''
    feature: 2D array (samples x features)
    labels: 1D boolean array (samples x)
    test_size: fraction of data to test on
    '''
    
    ### match GC content of samples labelled True with those labelled False by thowing out False samples
    # retrieve sequences using index of labels
    index_label_tuples = tuple(zip(labels.index.values, labels.values))
    
    true_ids = [x[0] for x in index_label_tuples if x[1]]
    
    false_ids = [x[0] for x in index_label_tuples if not x[1]]
       
    filtered_ids = true_ids + false_ids
    filtered_features = features[features.index.isin(filtered_ids)]
    filtered_labels = labels[labels.index.isin(filtered_ids)]

    if test_size <= 0.5:
        training_indices, test_indices = next(iter(
                sklearn.cross_validation.StratifiedKFold(filtered_labels, int(1/test_size), shuffle=True)))
    else:
        test_indices, training_indices = next(
            iter(sklearn.cross_validation.StratifiedKFold(filtered_labels, int(1/(1-test_size)), shuffle=True)))
    training_ids = [filtered_ids[i] for i in training_indices]
    test_ids = [filtered_ids[i] for i in test_indices]
    
    training_features = filtered_features[filtered_features.index.isin(training_ids)]
    test_features = filtered_features[filtered_features.index.isin(test_ids)]
    training_labels = filtered_labels[filtered_labels.index.isin(training_ids)]
    test_labels = filtered_labels[filtered_labels.index.isin(test_ids)]
    
    return training_features, test_features, training_labels, test_labels
    

## Copy Score Files

In [6]:
%%bash
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/motif_score_frame_C57BL6J.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/motif_sequence_frame_C57BL6J.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/motif_strand_frame_C57BL6J.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/motif_start_frame_C57BL6J.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/motif_end_frame_C57BL6J.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/summary_frame.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/annotation_frame.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/peak_sequences/C57BL6J.fa ./

## Read in Score Files

In [3]:
motif_score_frame=pd.read_pickle('motif_score_frame_C57BL6J.pickle')
motif_sequence_frame = pd.read_pickle('motif_sequence_frame_C57BL6J.pickle')
motif_strand_frame = pd.read_pickle('motif_strand_frame_C57BL6J.pickle')
motif_start_frame = pd.read_pickle('motif_start_frame_C57BL6J.pickle')
motif_end_frame = pd.read_pickle('motif_end_frame_C57BL6J.pickle')
summary_frame = pd.read_pickle('summary_frame.pickle')
annotation_frame = pd.read_pickle('annotation_frame.pickle')

scaler = preprocessing.MinMaxScaler()
normed_motif_frame = pd.DataFrame(scaler.fit_transform(motif_score_frame.ix[:,3:]))
normed_motif_frame.columns = motif_score_frame.columns.values[3:]
normed_motif_frame.index = motif_score_frame.index.values

scaler = preprocessing.StandardScaler()
standardized_motif_frame = pd.DataFrame(scaler.fit_transform(motif_score_frame.ix[:,3:]))
standardized_motif_frame.columns = motif_score_frame.columns.values[3:]
standardized_motif_frame.index = motif_score_frame.index.values

_factors = sorted(list(set([x.split('_')[1] for x in summary_frame.columns if '_' in x])))
_factors.remove('atac')

### read in sequences as dictionary {peakID: sequence}
with open('./C57BL6J.fa') as f:
    data = f.readlines()

_id_sequence_dict = {}
for line in data:
    if line[0] == '>':
        sequenceName = line.strip()[1:]
    else:
        _id_sequence_dict[sequenceName] = line.strip().upper()    

## Create background peaks from genomic sequences from each chromosome

In [6]:
# def getRandomBackgroundSwitch(target_positions, 
#                         size_ratio = 1.0, 
#                         tolerance = 0.05, 
#                         N_threshold = 0.5 ):
#     '''
#     target_sequences: 2D numpy array, list of genomic coordinates for target sequences [[chr,start,end],...]
#     size_ratio: float, number of background sequences relative to target sequences
#     tolerance: float, max difference in GC content between True and background labelled samples
#     *** Uses mm10 genome taken from Homer ***
#     '''
    
#     ###load mm10 genome into memory
    
#     # index target positions
#     # {chr:[]}, value is chromosome length boolean array
#     # largest chromosome has 200 million bps 
#     _chromosomes = ['chr1' , 'chr2' , 'chr3' , 'chr4' , 'chr5' , 
#                     'chr6' , 'chr7' , 'chr8' , 'chr9' , 'chr10', 
#                     'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 
#                     'chr16', 'chr17', 'chr18', 'chr19', 'chrX']
#     _chrom_size_dict = {}
#     _chrom_seq_dict = {}
#     for chrom in _chromosomes:
#         with open('./mm10_genome/' + chrom + '.fa') as f:
#             data = f.readlines()
#         seq = ''.join(x.upper().strip() for x in data[1:])
#         size = len(seq)
#         _chrom_size_dict[chrom] = size
#         _chrom_seq_dict[chrom] = seq
#     _numChromosomes = len(_chromosomes)
    
#     target_chr_position_dict = {x:np.zeros(200000000) for x in _chromosomes} 
#     ### initialize target_chr_position_dict using target positions
#     ### retreive target sequences
#     target_sequences = []
#     for pos in target_positions:
#         chrom = pos[0]        
#         start = pos[1]
#         end = pos[2]
#         target_chr_position_dict[chrom][start-1:end] = 1 # use 0 indexing of position, versus 1 indexing used in fasta
#         seq = _chrom_seq_dict[chrom][start:(end+1)]
#         target_sequences.append(seq)
#     ### calculate GC content and average length of the target sequences
#     target_gc_count = 0
#     target_length_count = 0
#     for s in target_sequences:
#         target_gc_count += s.count('G')
#         target_gc_count += s.count('C')
#         target_length_count += len(s)
#     target_gc_content = target_gc_count/(target_length_count+0.0000001) # GC content of target sequences
#     mean_target_length = target_length_count/len(target_sequences) # average length of target sequences
#     mean_target_length = int(mean_target_length)
    
#     ### select random genomic loci such that they do no overlap target sequences
#     numSelected = 0
#     numToSelect = len(target_positions) * size_ratio * 2 # candidate pool of background seqs is 2X larger
#     candidate_positions = []
#     while numSelected < numToSelect:
#         # select random chromsome
#         chromIndex = np.random.randint(_numChromosomes)
#         randChrom = _chromosomes[chromIndex]
#         randChromSize = _chrom_size_dict[randChrom]
#         # must find non overlapping segment on this chromosome before moving on
#         selectedSequence = False
#         while not selectedSequence:
#             randStart = np.random.randint(randChromSize)
#             randEnd = randStart + mean_target_length
#             overlap_sum = np.sum(target_chr_position_dict[randChrom][randStart:(randEnd + 1)])
#             if not overlap_sum > 0:
#                 selectedSequence = True
#                 numSelected+=1
#                 candidate_positions.append([randChrom, randStart, randEnd])

#     ### retrieve sequences of random genomic loci
#     numFiltered=0
#     filtered_candidate_positions = []
#     numNallowed = int(N_threshold * mean_target_length)
#     for cp in candidate_positions:
#         chrom = cp[0]
#         start = cp[1]
#         end = cp[2]
#         candidate_seq = _chrom_seq_dict[chrom][start:(end+1)]
#         numN = candidate_seq.count('N')
#         # throw away background peaks containing greater than this fraction of N
#         if numN <= numNallowed:
#             filtered_candidate_positions.append((chrom, start, end,candidate_seq))

#     if len(filtered_candidate_positions) < len(target_positions):
#         print('The genome is vast and empty and filled with Ns')
#         return None
       
#     ### select random set of candidate background loci
#     random.shuffle(filtered_candidate_positions)
    
#     toReturn_positions = filtered_candidate_positions[:len(target_positions)]
#     remaining_positions = filtered_candidate_positions[len(target_positions):]
#     # calcuate GC content of background samples
#     background_gc_count = 0
#     background_length = 0
#     for trp in toReturn_positions:
#         s = trp[3]
#         background_gc_count += s.count('G')
#         background_gc_count += s.count('C')
#         background_length += len(s)
#     background_gc_content = background_gc_count/(background_length+0.0000001)
        
#     numToReturn = len(toReturn_positions)
#     numRemaining = len(remaining_positions)
#     counter = 0
#     while abs(target_gc_content - background_gc_content) > tolerance:
#         # swith background GC sequences until GC content matches tolerance
#         switched_seq = False       
#         while not switched_seq:
#             # sequence to be switched out
#             rand_index = np.random.randint(numToReturn)
#             current_seq = toReturn_positions[rand_index][3]
#             current_gc_count = current_seq.count('G')+ current_seq.count('C')
#             current_length = len(current_seq)
#             current_gc = current_gc_count/current_length
            
#             # sequence to be switched out
#             switch_index = np.random.randint(numRemaining)
#             switch_seq = remaining_positions[switch_index][3]
#             switch_gc_count = switch_seq.count('G')+ switch_seq.count('C')
#             switch_length = len(switch_seq)
#             switch_gc = switch_gc_count/switch_length
#             if target_gc_content > background_gc_content:
#                 # switch sequences that would increase overall GC content of background sequences
#                 if switch_gc > current_gc:
#                     switched_seq = True
#             else:
#                 # switch sequences that would decrease overall GC content of background sequences
#                 if switch_gc < current_gc:
#                     switched_seq = True
#         counter +=1
#         if counter % 1000 == 0:
#             print(background_gc_content, target_gc_content, tolerance)
#         # switch sequences
#         temp_pos = toReturn_positions[rand_index]
#         toReturn_positions[rand_index] = remaining_positions[switch_index]
#         remaining_positions[switch_index] = temp_pos

#         # update background GC content
#         background_gc_count -= current_gc_count
#         background_length -= current_length
#         background_gc_count += switch_gc_count
#         background_length += switch_length
        
#         background_gc_content = background_gc_count/background_length
          
        
#     print(target_gc_content, background_gc_content)
#     return None
    

In [7]:
def getRandomBackground(target_positions, 
                        size_ratio = 1.0, 
                        tolerance = 0.01, 
                        N_threshold = 0.5 ):
    '''
    target_sequences: 2D numpy array, list of genomic coordinates for target sequences [[chr,start,end],...]
    size_ratio: float, number of background sequences relative to target sequences
    tolerance: float, max difference in GC content between True and background labelled samples
    *** Uses mm10 genome taken from Homer ***
    '''
    
    ###load mm10 genome into memory
    
    # index target positions
    # {chr:[]}, value is chromosome length boolean array
    # largest chromosome has 200 million bps 
    _chromosomes = ['chr1' , 'chr2' , 'chr3' , 'chr4' , 'chr5' , 
                    'chr6' , 'chr7' , 'chr8' , 'chr9' , 'chr10', 
                    'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 
                    'chr16', 'chr17', 'chr18', 'chr19', 'chrX']
    _chrom_size_dict = {}
    _chrom_seq_dict = {}
    for chrom in _chromosomes:
        with open('./mm10_genome/' + chrom + '.fa') as f:
            data = f.readlines()
        seq = ''.join(x.upper().strip() for x in data[1:])
        size = len(seq)
        _chrom_size_dict[chrom] = size
        _chrom_seq_dict[chrom] = seq
    _numChromosomes = len(_chromosomes)
    
    target_chr_position_dict = {x:np.zeros(200000000) for x in _chromosomes} 
    ### initialize target_chr_position_dict using target positions
    ### retreive target sequences
    target_sequences = []
    for pos in target_positions:
        chrom = pos[0]        
        start = pos[1]
        end = pos[2]
        target_chr_position_dict[chrom][start-1:end] = 1 # use 0 indexing of position, versus 1 indexing used in fasta
        seq = _chrom_seq_dict[chrom][start:(end+1)]
        target_sequences.append(seq)
    ### calculate GC content and average length of the target sequences
    target_gc_count = 0
    target_length_count = 0
    for s in target_sequences:
        target_gc_count += s.count('G')
        target_gc_count += s.count('C')
        target_length_count += len(s)
    target_gc_content = target_gc_count/(target_length_count+0.0000001) # GC content of target sequences
    mean_target_length = target_length_count/len(target_sequences) # average length of target sequences
    mean_target_length = int(mean_target_length)
    
    ### select random genomic loci such that they do no overlap target sequences
    numSelected = 0
    numToSelect = len(target_positions) * size_ratio * 2 # candidate pool of background seqs is 2X larger
    candidate_positions = []
    numNallowed = int(N_threshold * mean_target_length) # number of allowable Ns
    counter = 0
    while numSelected < numToSelect:
        if counter % 100000 == 0:
            print(counter, numSelected)
        # select random chromsome
        chromIndex = np.random.randint(_numChromosomes)
        randChrom = _chromosomes[chromIndex]
        randChromSize = _chrom_size_dict[randChrom]
        # must find non overlapping segment on this chromosome before moving on
        selectedSequence = False
        while not selectedSequence:
            counter += 1
            randStart = np.random.randint(randChromSize)
            randEnd = randStart + mean_target_length
            overlap_sum = np.sum(target_chr_position_dict[randChrom][randStart:(randEnd + 1)])
            
            if not overlap_sum > 0:
                randSeq = _chrom_seq_dict[randChrom][randStart:(randEnd+1)]
                numN = randSeq.count('N')
                if numN <= numNallowed:
                    rand_gc_count = randSeq.count('G')+ randSeq.count('C')
                    rand_gc = rand_gc_count/mean_target_length
                    if abs(target_gc_content - rand_gc) <= tolerance:
                        selectedSequence = True
                        numSelected+=1
                        candidate_positions.append([randChrom, randStart, randEnd, randSeq])
    # calcuate GC content of background samples
    background_gc_count = 0
    background_length = 0
    for cp in candidate_positions:
        s = cp[3]
        background_gc_count += s.count('G')
        background_gc_count += s.count('C')
        background_length += len(s)
    background_gc_content = background_gc_count/(background_length+0.0000001)
    print(target_gc_content,background_gc_content)
    return candidate_positions

In [234]:
# generate random genomic background for all monomers
strain = 'c57bl6'
for treatment in ['veh', 'kla']:
    for monomer in ap1_members:
        target_indices = summary_frame[summary_frame[strain + '_' + monomer + '_' + treatment] > 0.0].index.values
        target_positions = summary_frame[summary_frame.index.isin(target_indices)][['chr', 'start', 'end']].values
        start = time.time()
        backgroundPositions = getRandomBackground(target_positions, N_threshold =1.0, tolerance=0.05, size_ratio=5)
        end = time.time()
        print(monomer, treatment, end - start)
        pickle.dump(backgroundPositions,open('./background_pickles/' + monomer + '_' + treatment + '_background.pickle', 'wb'))

### Background positions to create peak files


In [26]:
strain = 'c57bl6'
! if [ ! -d ./background_peak_files ]; then mkdir ./background_peak_files; fi
for treatment in ['veh', 'kla']:
    for monomer in ap1_members:
        backgroundPositions = pickle.load(open('./background_pickles/' + monomer + '_' + treatment + '_background.pickle', 'rb'))
        outFile = open('./background_peak_files/' + strain + '_' + monomer + '_' + treatment + '-background_peaks.tsv' , 'w')
        outFile.write('\t'.join(['#PeakID','chr','start','end','strand','idrScore', 'count','\n']))
        counter = 0
        for pos in backgroundPositions:
            chrom = pos[0]
            start = str(pos[1])
            end = str(pos[2])
            strand = '+' # arbitrary - for compatibility with downstream scripts
            score = '1' # arbitrary - for compatibility with downstream scripts
            randID = 'background_' + str(np.random.randint(10000)) + '_' + str(counter)
            counter += 1
            outFile.write('\t'.join([randID, chrom, start, end, strand, score, score, '\n']))
        outFile.close()
        
    

### Merge Peak Files

In [27]:
%%capture
! mergePeaks -d given ./background_peak_files/*tsv > ./background_merged_peaks.tsv


In [28]:
! makeSummaryFile.py ./background_merged_peaks.tsv ./background_group_summary.tsv ./background_peak_files/*

sys:1: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
Finished reading merged peak file...
Integrating scores for c57bl6_atf3_kla-background
Integrating scores for c57bl6_atf3_veh-background
Integrating scores for c57bl6_cjun_kla-background
Integrating scores for c57bl6_cjun_veh-background
Integrating scores for c57bl6_fos_kla-background
Integrating scores for c57bl6_fos_veh-background
Integrating scores for c57bl6_junb_kla-background
Integrating scores for c57bl6_junb_veh-background
Integrating scores for c57bl6_jund_kla-background
Integrating scores for c57bl6_jund_veh-background


In [207]:
# read in peak data data
summary_background_frame = pd.read_csv('./background_group_summary.tsv' , sep='\t')
summary_background_frame = background_summary.fillna('0')
for col in summary_background_frame.columns[5:]:
    floatValues = []
    for val in summary_background_frame[col].values.astype(str):
        if ',' in val:
            maxVal = np.mean([float(x) for x in val.split(',')])
            floatValues.append(maxVal)
        else:
            floatValues.append(float(val))
    summary_background_frame[col] = floatValues
summary_background_frame.index = background_summary['ID'].values

/gpfs/data01/glasslab/home/jtao/software/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### create a script to scan for motifs using FIMO



In [29]:
%%bash
if [ ! -d ./peak_sequences ] ;
    then mkdir ./peak_sequences
else
    rm ./peak_sequences/*
fi

In [37]:
%%bash
perl /home/vlink/mouse_strains/marge/analysis/extract_seq_from_peakfiles.pl -strains C57BL6J -file ./background_merged_peaks.tsv -output ./peak_sequences/C57BL6J_marge.fa





Saving peaks
Loading shift vectors


In [39]:
# reformat fastq files to use homer peak IDs

coordinate_peakID_dict = {} # {chr_start_end:homerID}
with open ('./background_merged_peaks.tsv') as f:
    data = f.readlines()
for line in data[1:]:
    tokens = line.split('\t')
    coordinate = '_'.join(tokens[1:4])
    peakID = tokens[0].strip()
    coordinate_peakID_dict[coordinate] = tokens[0]



In [40]:
for fastaFile in os.listdir('./peak_sequences/'):
    if 'marge' in fastaFile:
        strain = fastaFile.split('_')[0]
        outFile = open('./peak_sequences/' + fastaFile.replace('_marge',''), 'w')
        print(fastaFile)
        with open('./peak_sequences/' + fastaFile) as f:
            data = f.readlines()
        for line in data:
            if '>' in line:
                coordinate = line[1:].replace('_'+strain,'').strip()
                
                peakID = coordinate_peakID_dict[coordinate]
                outFile.write('>' + peakID + '\n')
               
            else:
                outFile.write(line)
        outFile.close()

C57BL6J_marge.fa


In [63]:
# create a script to scan for motifs using FIMO
! if [ ! -d /home/jtao/analysis/random_background_analysis/fimo_results/ ]; then mkdir /home/jtao/analysis/random_background_analysis/fimo_results/; fi
! if [ ! -d /home/jtao/analysis/random_background_analysis/fimo_out/ ]; then mkdir /home/jtao/analysis/random_background_analysis/fimo_out/; fi
! rm -rf ./fimo_out/*
! rm -rf ./fimo_result/*


pthresh = 0.01
motif_dir = '/home/jtao/analysis/cobinding_motif_analysis/fimo_motifs/'

fimo_results_dir = './fimo_results'

for fastaFile in os.listdir('./peak_sequences/'):
    if not 'marge' in fastaFile:
        print(fastaFile)
        strain = fastaFile.split('.')[0]
        count = 0
        scriptFile = open('scanMotifs_background_'+ strain +'.sh','w')
        for m in os.listdir(motif_dir):
            if 'fimo' in m:
                fimo_out_dir = './fimo_out/' + strain + '_' +m.replace('.fimo','')
                outPath = fimo_results_dir + '/' +strain + '_' + m.replace('.fimo','') +'.txt'
                scriptFile.write(
                    '(sleep ' + str(15 * count) + 
                    's; fimo --text --max-stored-scores 2000000 --output-pthresh ' + 
                    str(pthresh) +' --oc ' + fimo_out_dir + ' ' +
                    motif_dir + '/' + m + ' ./peak_sequences/' + fastaFile +
                    '> ' + outPath + ' ) & \n')
                count+=1
        scriptFile.close()




C57BL6J.fa


In [ ]:
%%bash
chmod a+x ./scanMotifs*.sh
for i in ./scanMotifs*sh; 
    do echo 'sleeping...';
    echo $i;
    $i;
#     sleep 5m;
done



### Read in Motif Scores

In [70]:
from numba import jit

In [208]:
# @jit(nopython=False)
# def readMotifScores(fimoPaths, all_peak_ids):
#     '''
#     merges FIMO outputs for many motifs into a single data object
#     inputs: list of file paths to FIMO results
#     '''
    
#     ### initialize data structures for storing output
#     for fimo_result_path in fimoPaths:
#         toReturn = []
#         # read in fimo result file
#         motif_name = fimo_result_path.split('/')[-1].replace('.fimo','')
#         fimo_result_file = open(fimo_result_path, 'r')
#         fimo_result_data = fimo_result_file.readlines()
#         fimo_result_file.close()
        
#         # initialize arrays for storing current file
#         ids = []
#         scores = []
#         strands = []
#         sequences = []
#         starts = []
#         ends = []
#         fimo_result_data.pop(0) # skip header
#         for line in fimo_result_data: 
#             # break up row into column values
#             tokens = line.split('\t')
#             motif_name = tokens[0]
#             peak_id = tokens[1]
#             start = tokens[2]
#             stop = tokens[3]
#             strand = tokens[4]
#             score = tokens[5]
#             pvalue = tokens[6]
#             sequence = tokens[7]
            
#             ids.append(peak_id)
#             scores.append(score)
#             strands.append(strand)
#             sequences.append(sequence)
#             starts.append(start)
#             ends.append(end)        

#         id_count_dict = {} #{PeakID:motifCount}
#         id_values_dict = {} # {PeakID:(motifScore, motifSequence, motifStrand, motifStart, motifEnd)}

#         for i in range(len(ids)):
#             currentScore = float(scores[i])
#             currentSequence = sequences[i]
#             currentStrand = strands[i]
#             currentStart = int(starts[i])
#             currentEnd = int(ends[i])

#             if currentScore < 0.0:
#                 currentScore = 0.0


#             values = (currentScore, currentSequence, currentStrand, currentStart, currentEnd)

#             if ids[i] in id_values_dict:
#                 if currentScore > id_values_dict[ids[i]][0]:
#                     id_values_dict[ids[i]] = values
#                     id_count_dict[ids[i]] += 1
#             else:
#                 id_values_dict[ids[i]] = values
#                 id_count_dict[ids[i]] = 1
                
#         # fill in missing values
#         sorted_scores = []
#         sorted_sequences = []
#         sorted_strands = []
#         sorted_starts = []
#         sorted_ends = []
#         sorted_counts = []

#         for peak_id in all_peak_ids:
#             if peak_id in id_values_dict:
#                 current_values = id_values_dict[peak_id]
#                 current_score = current_values[0]
#                 current_sequence = current_values[1]
#                 current_strand = current_values[2]
#                 current_start = current_values[3]
#                 current_end = current_values[4]
#                 if peak_id in id_count_dict:
#                     current_count = id_count_dict[peak_id]
#                 else:
#                     current_count = 0
#             else:
#                 current_score = ''
#                 current_sequence = ''
#                 current_strand = '?'
#                 current_start = -1
#                 current_end = -1
#                 current_count = 0
#             sorted_scores.append(current_score)
#             sorted_sequences = [current_sequence]
#             sorted_strands = [current_strand]
#             sorted_starts = [current_start]
#             sorted_ends = [current_end]
#             sorted_counts = [current_count]
        

#         toReturn.append([sorted_scores,sorted_sequences,sorted_strands,sorted_starts,sorted_ends,sorted_counts])

    
#     return None
    
    
    

In [178]:
testPaths  = fimoPaths[:2]
all_peak_ids = summary_background_frame.index.values
readMotifScores(testPaths, all_peak_ids)

In [ ]:
motif_dir = '/home/jtao/analysis/cobinding_motif_analysis/fimo_motifs/'

for fastaFile in os.listdir('./peak_sequences/'):
    if not 'marge' in fastaFile:
        print(fastaFile)
        strain = fastaFile.split('.')[0]

        peak_start_dict = dict(zip(summary_background_frame['ID'].values, summary_background_frame['start'].values))

        motif_score_background_frame = summary_background_frame[['ID', 'Factors', 'chr']]
        motif_score_background_frame.index=motif_score_background_frame['ID'].values

        motif_sequence_background_frame = summary_background_frame[['ID', 'Factors', 'chr']]
        motif_sequence_background_frame.index=motif_score_background_frame['ID'].values

        motif_strand_background_frame = summary_background_frame[['ID', 'Factors', 'chr']]
        motif_strand_background_frame.index=motif_score_background_frame['ID'].values

        motif_start_background_frame = summary_background_frame[['ID', 'Factors', 'chr']]
        motif_start_background_frame.index=motif_score_background_frame['ID'].values

        motif_end_background_frame = summary_background_frame[['ID', 'Factors', 'chr']]
        motif_end_background_frame.index=motif_score_background_frame['ID'].values

        motif_count_background_frame = summary_background_frame[['ID', 'Factors', 'chr']]
        motif_count_background_frame.index = motif_count_background_frame['ID'].values

        counter=0
        for m in sorted(os.listdir(motif_dir)):
            counter+=1
            if '.fimo' in m:
                print(counter,m)
                motif_results = './fimo_results/' + strain + '_' + m.replace('.fimo','') +'.txt'
                fimo_result_background_frame=pd.read_csv(motif_results, 
                                              skiprows=1,
                                              names=['motif_name', 
                                                     'peak_id', 
                                                     'start', 
                                                     'stop', 
                                                     'strand', 
                                                     'score', 
                                                     'pvalue', 
                                                     'sequence'],
                                              sep='\t')
                motif_name = m.replace('.fimo','')

                id_count_dict = {} #{PeakID:motifCount}
                id_values_dict = {} # {PeakID:(motifScore, motifSequence, motifStrand, motifStart, motifEnd)}

                ids = fimo_result_background_frame['peak_id'].values
                scores = fimo_result_background_frame['score'].values
                strands = fimo_result_background_frame['strand'].values
                sequences = fimo_result_background_frame['sequence']
                starts = fimo_result_background_frame['start']
                ends = fimo_result_background_frame['stop']

                for i in range(len(ids)):
                    currentScore = float(scores[i])
                    currentSequence = sequences[i]
                    currentStrand = strands[i]
                    currentStart = int(starts[i])
                    currentEnd = int(ends[i])

                    if currentScore < 0.0:
                        currentScore = 0.0
                    
                    
                    values = (currentScore, currentSequence, currentStrand, currentStart, currentEnd)

                    if ids[i] in id_values_dict:
                        if currentScore > id_values_dict[ids[i]][0]:
                            id_values_dict[ids[i]] = values
                            id_count_dict[ids[i]] += 1
                    else:
                        id_values_dict[ids[i]] = values
                        id_count_dict[ids[i]] = 1

                sorted_values = [id_values_dict[x] if x in id_values_dict else (0,'','?',-1,-1,0) for x in  motif_score_background_frame['ID'].values]
                sorted_scores = [x[0] for x in sorted_values]
                sorted_sequences = [x[1] for x in sorted_values]
                sorted_strands = [x[2] for x in sorted_values]
                sorted_starts = [x[3] for x in sorted_values]
                sorted_ends = [x[4] for x in sorted_values]
                sorted_counts = [id_count_dict[x] if x in id_count_dict else 0 for x in motif_score_background_frame['ID'].values]

                motif_score_background_frame[motif_name] = sorted_scores
                motif_sequence_background_frame[motif_name] = sorted_sequences
                motif_strand_background_frame[motif_name] = sorted_strands
                motif_start_background_frame[motif_name] = sorted_starts
                motif_end_background_frame[motif_name] = sorted_ends
                motif_count_background_frame[motif_name] = sorted_counts


        motif_score_background_frame.to_pickle('motif_score_background_frame_'+  strain + '.pickle')
        motif_score_background_frame.to_csv('motif_scores_'+  strain + '.tsv', sep='\t', index=False)

        motif_sequence_background_frame.to_pickle('motif_sequence_background_frame_'+  strain + '.pickle')
        motif_sequence_background_frame.to_csv('motif_sequence_'+  strain + '.tsv', sep='\t', index=False)

        motif_strand_background_frame.to_pickle('motif_strand_background_frame_'+  strain + '.pickle')
        motif_strand_background_frame.to_csv('motif_strand_'+  strain + '.tsv', sep='\t', index=False)

        motif_start_background_frame.to_pickle('motif_start_background_frame_'+  strain + '.pickle')
        motif_start_background_frame.to_csv('motif_start_background_frame_'+  strain + '.tsv', sep='\t', index=False)

        motif_end_background_frame.to_pickle('motif_end_background_frame_'+  strain + '.pickle')
        motif_end_background_frame.to_csv('motif_end_background_frame_'+  strain + '.tsv', sep='\t', index=False)

        motif_count_background_frame.to_pickle('motif_count_background_frame_'+  strain + '.pickle')
        motif_count_background_frame.to_csv('motif_count_background_frame_'+  strain + '.tsv', sep='\t', index=False)



C57BL6J.fa
1 alx1_alx4_arx.fimo


/gpfs/data01/glasslab/home/jtao/software/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/gpfs/data01/glasslab/home/jtao/software/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/gpfs/data01/glasslab/home/jtao/software/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

2 ap-1.fimo


## Run Classifier using Open Chromatin Background

In [4]:
numIterations = 5
ap1_members = ['atf3','cjun', 'fos', 'junb','jund']    
test_size = 0.5
factors = ['atf3','cjun', 'fos', 'junb','jund', 'atac', 'cebpa', 'pu1', 'p65']
# c57bl6_indices = summary_frame[summary_frame['Factors'].str.contains('c57bl6')].index.values  



In [5]:

# for monomers using all motifs
strain = 'c57bl6'
factor_auc_dict = {}
factor_precision_dict = {}
factor_coeff_dict = {}
factor_prob_dict = {}
factor_meanCoeff_dict = {}
factor_intercept_dict = {}
factor_meanIntercept_dict = {}
for treatment in ['veh', 'kla']:
    for monomer in ap1_members:
        c57bl6_indices = summary_frame[summary_frame[['c57bl6_' + x + '_' + treatment for x in factors]].sum(axis=1) > 0].index.values  
        features = standardized_motif_frame[standardized_motif_frame.index.isin(c57bl6_indices)]
        labels = summary_frame[summary_frame.index.isin(c57bl6_indices)][strain + '_' + monomer + '_' + treatment] > 0.0
        if np.sum(labels) >= 100:
            all_aucs = []
            all_coefficients = []
            all_probs = None
            all_precisions = []
            all_intercepts = []
            for i in range(numIterations):  

                # split data into training and test sets
                training_features, test_features, training_labels, test_labels = get_GC_matched_split(
                    features, labels, test_size = test_size, tolerance = 0.01)

                #  Run classifier
                lr_classifier = sklearn.linear_model.LogisticRegression(penalty='l1', n_jobs=-1)

                lr_classifier.fit(training_features, training_labels)
                # retrieve probabilities
                probas_lr = lr_classifier.predict_proba(test_features)

                # score predictions
                current_roc_auc = sklearn.metrics.roc_auc_score(test_labels, probas_lr[:, 1], average = None)
                current_precision = sklearn.metrics.average_precision_score(test_labels, probas_lr[:, 1], average = None)

                all_aucs.append(current_roc_auc)
                all_precisions.append(current_precision)

                # score all sequences
                probs = lr_classifier.predict_proba(features)[:, 1]

                current_coefficients = lr_classifier.coef_.flatten()
                all_coefficients.append(current_coefficients)
                all_intercepts.append(lr_classifier.intercept_[0])
                
                if all_probs == None:
                    all_probs = probs
                else:
                    all_probs = all_probs + probs
            mean_coefficients = np.mean(all_coefficients, axis=0)
            
            factor_auc_dict[monomer + '_' + treatment]= all_aucs
            factor_precision_dict[monomer + '_' + treatment] = all_precisions
            factor_coeff_dict[monomer + '_' + treatment] = all_coefficients
            factor_prob_dict[monomer + '_' + treatment] = all_probs
            factor_meanCoeff_dict[monomer + '_' + treatment] = mean_coefficients
            factor_intercept_dict[monomer + '_' + treatment] = all_intercepts
            factor_meanIntercept_dict[monomer + '_' + treatment] = np.mean(all_intercepts)
            print(monomer + '_' + treatment,
                  'roc:', np.mean(all_aucs), np.var(all_aucs),
                  'precision:', np.mean(all_precisions), np.var(all_precisions),  
                  'numTestPositives:', np.sum(test_labels)
                 )

/gpfs/data01/glasslab/home/jtao/software/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:49: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


atf3_veh roc: 0.83610172069 3.44979507917e-06 precision: 0.710825327053 9.32621330292e-06 numTestPositives: 10969
cjun_veh roc: 0.812963450027 1.03597985076e-05 precision: 0.471024807596 5.39031299559e-06 numTestPositives: 6236
fos_veh roc: 0.860914986321 3.15643632627e-06 precision: 0.364336559092 6.13557879045e-05 numTestPositives: 992
junb_veh roc: 0.672269589368 6.21464145858e-05 precision: 0.0204829660345 2.08456526182e-06 numTestPositives: 263
jund_veh roc: 0.806859001358 2.04753834529e-06 precision: 0.563707505366 1.42685798554e-05 numTestPositives: 8885
atf3_kla roc: 0.830329033402 3.74700584789e-06 precision: 0.801807736212 5.36772708301e-06 numTestPositives: 17202
cjun_kla roc: 0.806956915332 5.27155667865e-07 precision: 0.479906804462 1.24805951856e-05 numTestPositives: 7994
fos_kla roc: 0.831760628236 2.70513489743e-06 precision: 0.662119375555 8.4607291233e-06 numTestPositives: 10745
junb_kla roc: 0.827412325119 1.60058468563e-06 precision: 0.479888666034 3.36324625181e-05

## Run Logistic Regression using Random Genomic Background

In [212]:
scaler = preprocessing.StandardScaler()
standardized_motif_background_frame = pd.DataFrame(scaler.fit_transform(motif_score_background_frame.ix[:,3:]))
standardized_motif_background_frame.columns = motif_score_background_frame.columns.values[3:]
standardized_motif_background_frame.index = motif_score_background_frame.index.values

In [ ]:
numIterations = 5
ap1_members = ['atf3','cjun', 'fos', 'junb','jund']    
test_size = 0.5
factors = ['atf3','cjun', 'fos', 'junb','jund', 'atac', 'cebpa', 'pu1', 'p65']
# c57bl6_indices = summary_frame[summary_frame['Factors'].str.contains('c57bl6')].index.values  



In [247]:
# for monomers using all motifs
strain = 'c57bl6'
factor_auc_randomBackground_dict = {}
factor_precision_randomBackground_dict = {}
factor_coeff_randomBackground_dict = {}
factor_prob_randomBackground_dict = {}
factor_meanCoeff_randomBackground_dict = {}
factor_intercept_randomBackground_dict = {}
factor_meanIntercept_randomBackground_dict = {}
for treatment in ['veh', 'kla']:
    for monomer in ap1_members:
        target_indices = summary_frame[summary_frame[strain + '_' + monomer + '_' + treatment] > 0.0].index.values
        target_features = standardized_motif_frame[standardized_motif_frame.index.isin(target_indices)]
        
        background_indices = motif_score_background_frame[
            motif_score_background_frame['Factors'] == strain + '_' + monomer + '_' + treatment + '-background'].index.values
        # select subset of background indices to use
        shuffle(background_indices)
        background_indices = background_indices[:5*len(target_indices)]
        background_features = standardized_motif_background_frame[standardized_motif_background_frame.index.isin(background_indices)]
        
        # merge target and background features together 
        features = pd.concat([target_features, background_features])
        labels = pd.Series(data = [True] * len(target_indices) + [False] * len(background_indices),
                           index = features.index.values)
        if np.sum(labels) >= 100:
            all_aucs = []
            all_coefficients = []
            all_probs = None
            all_precisions = []
            all_intercepts = []
            for i in range(numIterations):  

                # split data into training and test sets
                training_features, test_features, training_labels, test_labels = get_split(
                    features, labels, test_size = test_size)

                #  Run classifier
                lr_classifier = sklearn.linear_model.LogisticRegression(penalty='l1', n_jobs=-1)

                lr_classifier.fit(training_features, training_labels)
                # retrieve probabilities
                probas_lr = lr_classifier.predict_proba(test_features)

                # score predictions
                current_roc_auc = sklearn.metrics.roc_auc_score(test_labels, probas_lr[:, 1], average = None)
                current_precision = sklearn.metrics.average_precision_score(test_labels, probas_lr[:, 1], average = None)

                all_aucs.append(current_roc_auc)
                all_precisions.append(current_precision)

                # score all sequences
                probs = lr_classifier.predict_proba(features)[:, 1]

                current_coefficients = lr_classifier.coef_.flatten()
                all_coefficients.append(current_coefficients)
                all_intercepts.append(lr_classifier.intercept_[0])
                
                if all_probs == None:
                    all_probs = probs
                else:
                    all_probs = all_probs + probs
            mean_coefficients = np.mean(all_coefficients, axis=0)
            
            factor_auc_randomBackground_dict[monomer + '_' + treatment]= all_aucs
            factor_precision_randomBackground_dict[monomer + '_' + treatment] = all_precisions
            factor_coeff_randomBackground_dict[monomer + '_' + treatment] = all_coefficients
            factor_prob_randomBackground_dict[monomer + '_' + treatment] = all_probs
            factor_meanCoeff_randomBackground_dict[monomer + '_' + treatment] = mean_coefficients
            factor_intercept_randomBackground_dict[monomer + '_' + treatment] = all_intercepts
            factor_meanIntercept_randomBackground_dict[monomer + '_' + treatment] = np.mean(all_intercepts)
            print(monomer + '_' + treatment,
                  'roc:', np.mean(all_aucs), np.var(all_aucs),
                  'precision:', np.mean(all_precisions), np.var(all_precisions),  
                  'numTestPositives:', np.sum(test_labels)
                 )

/gpfs/data01/glasslab/home/jtao/software/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:59: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


atf3_veh roc: 0.917402357455 1.44864536324e-06 precision: 0.777096713825 6.70150610575e-06 numTestPositives: 11013
cjun_veh roc: 0.935315493846 6.80897403591e-07 precision: 0.827533261605 3.33182618986e-06 numTestPositives: 6335
fos_veh roc: 0.946941499214 1.4454350912e-05 precision: 0.83027582052 4.59802544223e-05 numTestPositives: 994
junb_veh roc: 0.83173455127 8.50686578456e-05 precision: 0.698923256979 0.000326516124148 numTestPositives: 247
jund_veh roc: 0.915915331233 6.30124727043e-07 precision: 0.779518635626 3.20334833347e-06 numTestPositives: 9025
atf3_kla roc: 0.913409716568 5.01326610535e-07 precision: 0.727766399839 8.07884603255e-06 numTestPositives: 17327
cjun_kla roc: 0.937144708727 8.33440018159e-07 precision: 0.817832503968 4.44897412885e-06 numTestPositives: 8068
fos_kla roc: 0.940092995887 3.37132354601e-07 precision: 0.80862130008 2.90010220952e-06 numTestPositives: 10733
junb_kla roc: 0.94854589066 5.01028336266e-07 precision: 0.851497590155 4.54380376101e-06 num

## Run gkmSVM against Random Genomic Background

In [ ]:
monomers = []
aucs = []
feature_set = []
for factor in sorted(factor_auc_dict_lr.keys()):
    monomers = monomers + numIterations * [factor]
    aucs = aucs + factor_auc_dict_lr[factor]
    feature_set = feature_set + numIterations * ['LR All Motifs']
    
    monomers = monomers + numIterations * [factor]
    aucs = aucs + factor_auc_dict_unbalancedgkmSVM[factor]
    feature_set = feature_set + numIterations * ['gkmSVM']
    
data = pd.DataFrame({'Factor':monomers,
                     'AUC':aucs,
                     'Feature Set': feature_set
                     })

# vehicle plot


for treatment in ['veh', 'kla']:
    factor_auc_tuples = [(x, np.mean(factor_auc_dict_lr[x])) for x in factor_auc_dict_lr if treatment in x]
    sorted_monomers = [y[0] for y in sorted(factor_auc_tuples, key=lambda x:x[1])]
    with sns.axes_style('ticks'):
        plottingFrame = sns.factorplot(data = data[data['Factor'].str.contains(treatment)],
                                    x='Factor',
                                    y='AUC',
                                    order = sorted_monomers,
                                    palette='Set1',
                                    size=4,
                                    hue='Feature Set',
                                    kind = 'point', 
                                    markers = '.', s='0.01')
        sns.despine()
        plt.xticks(rotation=90)
        plt.ylabel('aucROC')
        plt.xlabel('AP-1 Monomer')
        plt.ylim(0.5,1)
        plt.title('Classifier Performance')
        plt.show()

    